In [ ]:
!pip install ultralytics

In [1]:
!python3 -m pip install paddlepaddle==3.1.0 -i https://www.paddlepaddle.org.cn/packages/stable/cpu/
!python3 -m pip install paddlepaddle-gpu==3.1.0 -i https://www.paddlepaddle.org.cn/packages/stable/cu126/
!pip install paddleocr

Looking in indexes: https://www.paddlepaddle.org.cn/packages/stable/cpu/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.0/195.0 MB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 5.2 MB/s eta 0:00:00
  Attempting uninstall: opt_einsum
    Found existing installation: opt_einsum 3.4.0
    Uninstalling opt_einsum-3.4.0:
      Successfully uninstalled opt_einsum-3.4.0
Looking in indexes: https://www.paddlepaddle.org.cn/packages/stable/cu126/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 GB 834.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Uninstall conflicting PaddlePaddle versions
!pip uninstall -y paddlepaddle paddlepaddle-gpu

# Reinstall the version compatible with PaddleOCR (replace with the correct version if needed)
!pip install -q paddlepaddle-gpu==2.6.1 -f https://www.paddlepaddle.org.cn/whl/cu113/avx/stable.html
!pip install paddleocr
!pip install paddlex==3.0.0b2

Found existing installation: paddlepaddle-gpu 2.6.1
Uninstalling paddlepaddle-gpu-2.6.1:
  Successfully uninstalled paddlepaddle-gpu-2.6.1
  Using cached paddlex-3.1.2-py3-none-any.whl.metadata (78 kB)
  Using cached langchain-0.3.26-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain_community-0.3.27-py3-none-any.whl.metadata (2.9 kB)
  Using cached langchain_core-0.3.68-py3-none-any.whl.metadata (5.8 kB)
  Using cached langsmith-0.4.5-py3-none-any.whl.metadata (15 kB)
  Using cached numpy-2.3.1-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (62 kB)
Using cached paddlex-3.1.2-py3-none-any.whl (1.7 MB)
Using cached langchain-0.3.26-py3-none-any.whl (1.0 MB)
Using cached langchain_community-0.3.27-py3-none-any.whl (2.5 MB)
Using cached langchain_core-0.3.68-py3-none-any.whl (441 kB)
Using cached numpy-2.3.1-cp311-cp311-manylinux_2_28_x86_64.whl (16.9 MB)
Using cached langsmith-0.4.5-py3-none-any.whl (367 kB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.

  Using cached paddlex-3.0.0b2-py3-none-any.whl.metadata (52 kB)
  Using cached numpy-1.24.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
  Using cached langchain-0.1.5-py3-none-any.whl.metadata (13 kB)
  Using cached langchain_community-0.0.17-py3-none-any.whl.metadata (7.9 kB)
  Using cached langchain_core-0.1.53-py3-none-any.whl.metadata (5.9 kB)
  Using cached langsmith-0.0.92-py3-none-any.whl.metadata (9.9 kB)
INFO: pip is looking at multiple versions of langchain-core to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_core-0.1.52-py3-none-any.whl.metadata (5.9 kB)
  Using cached langchain_core-0.1.51-py3-none-any.whl.metadata (5.9 kB)
  Using cached langchain_core-0.1.50-py3-none-any.whl.metadata (5.9 kB)
  Using cached langchain_core-0.1.49-py3-none-any.whl.metadata (5.9 kB)
  Using cached langchain_core-0.1.48-py3-none-any.whl.metadata (5.9 kB)
  Using cached langchain_core-0.1.47-p

In [3]:
!pip install --quiet vietocr

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.0/948.0 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 99.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.6/299.6 kB 23.7 MB/s eta 0:00:00


In [ ]:
# from ultralytics import YOLO
import os
import matplotlib.pyplot as plt
from PIL import Image
from paddleocr import TextDetection
import cv2
import numpy as np

In [ ]:
# Chuyển các file pdf trong folder sang ảnh


# Folder paths
input_folder = '/content/drive/MyDrive/sample_intern1/sample'
pdf_output_folder = '/content/drive/MyDrive/sample_intern1/output/pdf_first_pages'
img_output_folder = '/content/drive/MyDrive/sample_intern1/output/img_first_pages'
os.makedirs(pdf_output_folder, exist_ok=True)
os.makedirs(img_output_folder, exist_ok=True)

# Loop through PDF files
for filename in os.listdir(input_folder):
    if filename.lower().endswith('.pdf'):
        file_path = os.path.join(input_folder, filename)
        base_name = os.path.splitext(filename)[0]

        # Extract first page and save as new PDF
        reader = PdfReader(file_path)
        writer = PdfWriter()
        if len(reader.pages) > 0:
            writer.add_page(reader.pages[0])
            pdf_output_path = os.path.join(pdf_output_folder, f'{base_name}_page1.pdf')
            with open(pdf_output_path, 'wb') as out_pdf:
                writer.write(out_pdf)
            print(f'✅ Saved first page: {pdf_output_path}')

            # Convert first page to image
            images = convert_from_path(pdf_output_path, first_page=1, last_page=1)
            img_output_path = os.path.join(img_output_folder, f'{base_name}_page1.jpg')
            images[0].save(img_output_path, 'JPEG')
            print(f'🖼️ Converted to image: {img_output_path}')


✅ Saved first page: /content/drive/MyDrive/sample_intern1/output/pdf_first_pages/hp-01_page1.pdf
🖼️ Converted to image: /content/drive/MyDrive/sample_intern1/output/img_first_pages/hp-01_page1.jpg


✅ Saved first page: /content/drive/MyDrive/sample_intern1/output/pdf_first_pages/lenh-01_page1.pdf
🖼️ Converted to image: /content/drive/MyDrive/sample_intern1/output/img_first_pages/lenh-01_page1.jpg
✅ Saved first page: /content/drive/MyDrive/sample_intern1/output/pdf_first_pages/luat-01_page1.pdf


🖼️ Converted to image: /content/drive/MyDrive/sample_intern1/output/img_first_pages/luat-01_page1.jpg
✅ Saved first page: /content/drive/MyDrive/sample_intern1/output/pdf_first_pages/luat-02_page1.pdf


🖼️ Converted to image: /content/drive/MyDrive/sample_intern1/output/img_first_pages/luat-02_page1.jpg
✅ Saved first page: /content/drive/MyDrive/sample_intern1/output/pdf_first_pages/luat-03_page1.pdf


🖼️ Converted to image: /content/drive/MyDrive/sample_intern1/output/img_first_pages/luat-03_page1.jpg
✅ Saved first page: /content/drive/MyDrive/sample_intern1/output/pdf_first_pages/luat-04_page1.pdf
🖼️ Converted to image: /content/drive/MyDrive/sample_intern1/output/img_first_pages/luat-04_page1.jpg
✅ Saved first page: /content/drive/MyDrive/sample_intern1/output/pdf_first_pages/luat-05_page1.pdf


...

🖼️ Converted to image: /content/drive/MyDrive/sample_intern1/output/img_first_pages/nd-05_page1.jpg
✅ Saved first page: /content/drive/MyDrive/sample_intern1/output/pdf_first_pages/nqcp-01_page1.pdf


🖼️ Converted to image: /content/drive/MyDrive/sample_intern1/output/img_first_pages/nqcp-01_page1.jpg
✅ Saved first page: /content/drive/MyDrive/sample_intern1/output/pdf_first_pages/nqcp-02_page1.pdf
🖼️ Converted to image: /content/drive/MyDrive/sample_intern1/output/img_first_pages/nqcp-02_page1.jpg
✅ Saved first page: /content/drive/MyDrive/sample_intern1/output/pdf_first_pages/pl-01_page1.pdf


🖼️ Converted to image: /content/drive/MyDrive/sample_intern1/output/img_first_pages/pl-01_page1.jpg
✅ Saved first page: /content/drive/MyDrive/sample_intern1/output/pdf_first_pages/pl-02_page1.pdf
🖼️ Converted to image: /content/drive/MyDrive/sample_intern1/output/img_first_pages/pl-02_page1.jpg
✅ Saved first page: /content/drive/MyDrive/sample_intern1/output/pdf_first_pages/pl-03_page1.pdf
🖼️ Converted to image: /content/drive/MyDrive/sample_intern1/output/img_first_pages/pl-03_page1.jpg
✅ Saved first page: /content/drive/MyDrive/sample_intern1/output/pdf_first_pages/pl-04_page1.pdf


🖼️ Converted to image: /content/drive/MyDrive/sample_intern1/output/img_first_pages/pl-04_page1.jpg
✅ Saved first page: /content/drive/MyDrive/sample_intern1/output/pdf_first_pages/qd-01_page1.pdf


🖼️ Converted to image: /content/drive/MyDrive/sample_intern1/output/img_first_pages/qd-01_page1.jpg
✅ Saved first page: /content/drive/MyDrive/sample_intern1/output/pdf_first_pages/qd-02_page1.pdf
🖼️ Converted to image: /content/drive/MyDrive/sample_intern1/output/img_first_pages/qd-02_page1.jpg
✅ Saved first page: /content/drive/MyDrive/sample_intern1/output/pdf_first_pages/qd-03_page1.pdf
🖼️ Converted to image: /content/drive/MyDrive/sample_intern1/output/img_first_pages/qd-03_page1.jpg
✅ Saved first page: /content/drive/MyDrive/sample_intern1/output/pdf_first_pages/qd-04_page1.pdf
🖼️ Converted to image: /content/drive/MyDrive/sample_intern1/output/img_first_pages/qd-04_page1.jpg
✅ Saved first page: /content/drive/MyDrive/sample_intern1/output/pdf_first_pages/qd-05_page1.pdf
🖼️ Converted to image: /content/drive/MyDrive/sample_intern1/output/img_first_pages/qd-05_page1.jpg


In [ ]:
# Train model YOLO để crop tiêu đề các văn bản

# Load a model
model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)

# Train the model
results = model.train(data="/content/drive/MyDrive/sample_intern1/dataset.yaml", epochs=100, imgsz=640)

100%|██████████| 6.25M/6.25M [00:00<00:00, 140MB/s]


Ultralytics 8.3.161 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/sample_intern1/dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True

100%|██████████| 755k/755k [00:00<00:00, 36.2MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

Model summary: 129 layers, 3,011,043 parameters, 3,011,027 gradients, 8.2 GFLOPs

Transferred 319/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


...

    100/100      2.28G      0.616      1.085     0.8961          7        640: 100%|██████████| 2/2 [00:00<00:00,  5.25it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  4.55it/s]

                   all         23         23      0.997          1      0.995       0.89



100 epochs completed in 0.036 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.161 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  5.65it/s]


                   all         23         23      0.997          1      0.995       0.89
Speed: 0.2ms preprocess, 1.4ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to runs/detect/train


In [ ]:
# Load pretrained model
model = YOLO("/content/drive/MyDrive/sample_intern1/best.pt")  # load a pretrained model (recommended for training)

# Train the model
results = model.train(data="/content/drive/MyDrive/sample_intern1/dataset.yaml", epochs=100, imgsz=640)

Ultralytics 8.3.161 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/sample_intern1/dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/drive/MyDrive/sample_intern1/best.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patie

train: Scanning /content/drive/MyDrive/sample_intern1/output/img_first_pages.cache... 23 images, 0 backgrounds, 0 corrupt: 100%|██████████| 23/23 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.5±0.3 ms, read: 144.2±44.3 MB/s, size: 486.3 KB)


val: Scanning /content/drive/MyDrive/sample_intern1/output/img_first_pages.cache... 23 images, 0 backgrounds, 0 corrupt: 100%|██████████| 23/23 [00:00<?, ?it/s]


...

    100/100      2.32G     0.5284     0.6811      0.873          7        640: 100%|██████████| 2/2 [00:00<00:00,  5.41it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  5.05it/s]

                   all         23         23      0.997          1      0.995      0.932



100 epochs completed in 0.034 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 6.2MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics 8.3.161 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  5.87it/s]


                   all         23         23      0.998          1      0.995      0.936
Speed: 0.2ms preprocess, 1.4ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to runs/detect/train2


In [ ]:
# CROP TỪNG ẢNH TRONG FOLDER 

# Đường dẫn đến thư mục chứa ảnh
image_folder = "/content/drive/MyDrive/sample_intern1/output/img_first_pages"
output_dir = "/content/drive/MyDrive/sample_intern1/output/test/"
os.makedirs(output_dir, exist_ok=True)

# Tải mô hình YOLO
model_crop = YOLO("/content/drive/MyDrive/sample_intern1/best.pt")

# Biến để đánh số thứ tự các ảnh crop
crop_count = 0

# Lặp qua tất cả các file ảnh trong thư mục
for filename in os.listdir(image_folder):
    # Kiểm tra xem file có phải là file ảnh không (jpg, jpeg, png, ...)
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        # Đường dẫn đầy đủ đến ảnh
        image_path = os.path.join(image_folder, filename)

        # Đọc ảnh
        image = cv2.imread(image_path)

        # Dự đoán trên ảnh
        results = model_crop(image)

        # Duyệt qua tất cả các kết quả dự đoán
        for result in results:
            # Duyệt qua từng box (bounding box) trong các kết quả dự đoán
            for box in result.boxes:
                # Lấy tọa độ của bounding box (x1, y1, x2, y2)
                x1, y1, x2, y2 = map(int, box.xyxy[0])

                # Trích xuất vùng ảnh trong bounding box
                object_image = image[y1:y2, x1:x2]

                # # Tạo tên file với số thứ tự
                output_filename = os.path.join(output_dir, f"crop_{crop_count}.jpg")

                # Lưu ảnh crop với tên file theo thứ tự
                cv2.imwrite(output_filename, object_image)

                # Tăng biến đếm
                crop_count += 1

                print(f"Đã lưu {output_filename}")


Using official model (PP-OCRv5_server_det), the model files will be automatically downloaded and saved in /root/.paddlex/official_models.


AttributeError: 'paddle.base.libpaddle.AnalysisConfig' object has no attribute 'set_optimization_level'

In [ ]:
# Text detection (PaddleOCR) và text recognition (vietocr)


from paddleocr import PaddleOCR
from vietocr.tool.predictor import Predictor
from vietocr.tool.config import Cfg
from PIL import Image, ImageDraw, ImageFont
import os
import numpy as np

# Khởi tạo PaddleOCR chỉ dùng text detection
ocr = PaddleOCR(use_angle_cls=False, lang='vi', text_detection_model_name="PP-OCRv5_mobile_det")

# Khởi tạo VietOCR cho text recognition
config = Cfg.load_config_from_name('vgg_transformer')
config['cnn']['pretrained'] = False
config['device'] = 'cuda:0'
vietocr_predictor = Predictor(config)

# Đường dẫn ảnh đầu vào
img_path = '/content/drive/MyDrive/sample_intern1/output/crops/crop_17.jpg'
image = Image.open(img_path).convert('RGB')
draw = ImageDraw.Draw(image)

# Font để vẽ text
try:
    font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", 18)
except:
    font = ImageFont.load_default()

recognized_text_list = []
# Tạo thư mục output nếu chưa có
os.makedirs("output/crops", exist_ok=True)

# Text detection bằng PaddleOCR
result = ocr.ocr(img_path)

# Lấy dt_polys từ kết quả
if result and isinstance(result[0], dict) and 'dt_polys' in result[0]:
    for idx, polygon in enumerate(result[0]['dt_polys']):
        # polygon là numpy array (4, 2), convert thành list point tuple
        box_coords = [tuple(map(int, pt)) for pt in polygon]

        # Tìm bounding box
        xs = [pt[0] for pt in box_coords]
        ys = [pt[1] for pt in box_coords]
        x_min, x_max = min(xs), max(xs)
        y_min, y_max = min(ys), max(ys)

        # Crop vùng ảnh
        cropped = image.crop((x_min, y_min, x_max, y_max))

        # Lưu ảnh crop
        crop_path = f"output/crops/crop_{idx}.jpg"
        cropped.save(crop_path)

        # Nhận dạng văn bản bằng VietOCR
        try:
            recognized_text = vietocr_predictor.predict(cropped)
            recognized_text_list.append(recognized_text)
        except:
            recognized_text = "[ERROR]"

        # Vẽ polygon + text lên ảnh gốc
        draw.line(box_coords + [box_coords[0]], fill=(255, 0, 0), width=2)
        draw.text((x_min, y_min - 10), recognized_text, fill=(0, 0, 255), font=font)

# Lưu ảnh kết quả
image.save("output/result_vietocr.jpg")


/tmp/ipython-input-29-3976656302.py:9: UserWarning: `lang` and `ocr_version` will be ignored when model names or model directories are not `None`.
  ocr = PaddleOCR(use_angle_cls=False, lang='vi', text_detection_model_name="PP-OCRv5_mobile_det")
/tmp/ipython-input-29-3976656302.py:9: DeprecationWarning: The parameter `use_angle_cls` has been deprecated and will be removed in the future. Please use `use_textline_orientation` instead.
  ocr = PaddleOCR(use_angle_cls=False, lang='vi', text_detection_model_name="PP-OCRv5_mobile_det")
Creating model: ('PP-LCNet_x1_0_doc_ori', None)
Using official model (PP-LCNet_x1_0_doc_ori), the model files will be automatically downloaded and saved in /root/.paddlex/official_models.


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Creating model: ('UVDoc', None)
Using official model (UVDoc), the model files will be automatically downloaded and saved in /root/.paddlex/official_models.


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Creating model: ('PP-OCRv5_mobile_det', None)
Using official model (PP-OCRv5_mobile_det), the model files will be automatically downloaded and saved in /root/.paddlex/official_models.


...

Creating model: ('PP-OCRv5_server_rec', None)
Using official model (PP-OCRv5_server_rec), the model files will be automatically downloaded and saved in /root/.paddlex/official_models.


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Model weight /tmp/vgg_transformer.pth exsits. Ignore download!


/tmp/ipython-input-29-3976656302.py:33: DeprecationWarning: Please use `predict` instead.
  result = ocr.ocr(img_path)


In [30]:
recognized_text_list

['PHÁP LỆNH',
 'SỬA ĐỒI, BỒ SUNG MỘT SỐ ĐIỀU CỦA',
 'PHÁP LỆNH QUẢN LÝ, SỬ DỤNG VŨ KHÍ, VẬT LIỆU NỒ',
 'VÀ CÔNG CỤ HỒ TRỢ']